库 word2vec、Gensim

数据源：[搜狐新闻数据(SogouCS)](http://www.sogou.com/labs/resource/cs.php)

In [1]:
import numpy as np
import pandas as pd
import jieba

In [2]:
df_news = pd.read_csv('data/xinwen.csv', encoding='utf-8')
df_news = df_news.dropna()
df_news.head(10)

,url,docno,title,content
0,http://sports.sohu.com/20070422/n249599819.shtml,0be9f205fcdf9e72,鲍威尔:中国队表现出乎意料 短距离配合很出色,"新华网武汉4月21日电(记者郑道锦 李鹏翔)在2:3输给中国女足(中国女足新闻,中国女足说吧..."
1,http://sports.sohu.com/20070422/n249599820.shtml,04f53205fcdf9e72,英超-孙继海轻伤不下火线 曼城1-1惜平沃特福德,"搜狐体育讯 北京时间4月21日晚22点,在英超联赛第35轮的一场比赛中,曼城客场迎战联赛倒数..."
2,http://sports.sohu.com/20070422/n249599822.shtml,02636525fcdf9e72,亚冠定律首度失灵 图拨炮轰洋裁判左右比赛进程,"搜狐体育讯 北京时间4月21日济南消息 同长春亚泰比赛之前,本赛季的山东鲁能一直存在一种神奇..."
3,http://sports.sohu.com/20070422/n249599831.shtml,0b3447c5fcdf9e72,搜狐-丁俊晖有约4:实话实说我的竞技状态一般,"搜狐体育讯 作为丁俊晖(丁俊晖博客,丁俊晖说吧)全球官方网站以及个人官方博客的承建商,搜狐在..."
4,http://sports.sohu.com/20070422/n249599835.shtml,0c16ae85fcdf9e72,英超-伟大战役查尔顿1-1谢联 郑智未闪耀生死战,"搜狐体育讯 北京时间4月21日22:00,在英超联赛第35轮的一场焦点战役中,查尔顿在山谷球..."
5,http://sports.sohu.com/20070422/n249599888.shtml,01f69205fcdf9e72,荷兰神枪双球贺喜得贵子 梅开二度许愿欧冠破荒,"搜狐体育讯 雪藏杰拉德的利物浦(利物浦新闻,利物浦说吧)坐镇主场兵不血刃地击败维冈竞技,荷兰..."
6,http://sports.sohu.com/20070422/n249599891.shtml,06b2aa35fcdf9e72,"前瞻-丁俊晖""火箭""恩怨再续 仅一人能继续争冠","搜狐体育讯 北京时间4月21日,2007斯诺克世锦赛(斯诺克世锦赛说吧)开始了第一天争夺。中..."
7,http://sports.sohu.com/20070422/n249599893.shtml,0ac20d05fcdf9e72,组图:库伊特送红军进前四 利物浦2-0维冈,"[点击图片进入下一页] picture 北京时间4月21日晚,英超第35轮全面开战。 红军利..."
8,http://sports.sohu.com/20070422/n249599955.shtml,03e62ea5fcdf9e72,联合会杯-彭帅错失赛点伤退 中国0-2落后意大利,"搜狐体育讯 北京时间4月22日凌晨,2007年联合会杯女子网球(网球新闻,网球说吧)团体赛世..."
9,http://sports.sohu.com/20070422/n249599989.shtml,032dd7e5fcdf9e72,图文:[英超]查尔顿1-1谢联 郑智迎战蒙哥马利,"搜狐体育讯 北京时间4月21日22:00,在英超联赛第35轮的一场焦点战役中,查尔顿在山谷球..."


In [3]:
df_news.shape

(953, 4)

### 分词：使用结巴分词器

In [4]:
content = df_news.content.values.tolist()
print(content[952])

搜狐体育讯 北京时间4月22日晚,荷甲联赛第33轮全面开打,领头羊埃因霍温在客场1-1被实力有限的乌特勒支逼平。与此同时,阿贾克斯以5-2的大比分击败了鹿特丹斯巴达,阿尔克马尔也以3-1战胜了海伦芬,这样3支球队同积72分,阿尔克马尔净胜球最多排在第一,阿贾克斯次之,埃因霍温被挤到了第三。 联赛只剩下最后一轮,埃因霍温就算在主场战胜维特斯,能否夺冠,也得看别人脸色,只要另外两队有一支队伍获胜,联赛冠军就将拱手相让。客观的说,埃因霍温夺取联赛冠军的希望已经变得渺茫。 中国球员孙祥本场比赛没能进入球队的18人比赛大名单,这也是孙祥自从2月18日代表球队首次亮相荷甲赛场以来,第一次未能进入球队的比赛大名单。此前的三轮比赛,尽管孙祥也未能出场,但是都进入18人比赛名单。在埃因霍温夺冠前景变得几近渺茫的同时,孙祥在埃因霍温的处境也进入最低谷。 在上一轮主场2-0战胜特温特的比赛后,主教练科曼在接受媒体采访,被问及孙祥下赛季去留问题的时候,他曾经肯定的表示,“本赛季如果埃因霍温能夺冠,孙祥就会留下。”可是1-1客场被乌特勒支逼平后,埃因霍温的领先优势荡然无存,而且埃因霍温现在比排名第一阿尔克马尔少了7个净胜球,要想在最后一轮追回来,谈何容易。埃因霍温如果夺冠,孙祥就会留下来,那么埃因霍温如果不能夺冠呢?孙祥下个赛季将何去何从。回顾最近一个月埃因霍温的比赛,孙祥代表球队出场的机会屈指可数,除了冠军杯的两次替补出场,荷甲联赛竟然没有得到一次出场机会,这场比赛更是连18人大名单都没进入。孙祥在埃因霍温的前景实在不容乐观。 从2月18日首次代表球队荷甲亮相就以首发的身份登场,到冠军联赛客场对阿森纳的首发打满全场,并制造关键的前场任意球,再到荷甲联赛的连续三轮首发,孙祥在埃因霍温的“蜜月期”来得很早也很“甜蜜”。但是一次比赛中膝盖的受伤让孙祥的状态受到很大的影响,连科曼都对其下滑的状态颇有微辞。在后防线最缺兵少将的时候,科曼无奈只好启用“千年板凳”阿多,而这位近几个赛季出场次数都寥寥无几的加纳边缘国脚,很好的把握住了机会,迅速占据了后防线上的一个主力位置,近几轮比赛场场首发,孙祥只能在替补席上与同样状态低迷的葡萄牙国脚达·科斯塔一起等待机会。 此前三轮联赛,尽管都未能出场,但是孙祥至少还是被召入了18人的比赛大名单。本轮联赛前,埃因霍温后防大将阿莱士的伤愈,将原先首发阵容里的阿尔西德斯挤

In [5]:
content_S = []
for line in content:
    current_segment = jieba.lcut(line)
    if len(current_segment) > 1 and current_segment != '\r\n':
        content_S.append(current_segment)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\cj_zu\AppData\Local\Temp\jieba.cache
Loading model cost 0.918 seconds.
Prefix dict has been built succesfully.


In [6]:
content_S[952]

['搜狐',
 '体育讯',
 ' ',
 '北京',
 '时间',
 '4',
 '月',
 '22',
 '日晚',
 ',',
 '荷甲',
 '联赛',
 '第',
 '33',
 '轮',
 '全面',
 '开打',
 ',',
 '领头羊',
 '埃因霍温',
 '在',
 '客场',
 '1',
 '-',
 '1',
 '被',
 '实力',
 '有限',
 '的',
 '乌',
 '特勒',
 '支逼平',
 '。',
 '与此同时',
 ',',
 '阿贾克斯',
 '以',
 '5',
 '-',
 '2',
 '的',
 '大比分',
 '击败',
 '了',
 '鹿特丹',
 '斯巴达',
 ',',
 '阿尔克',
 '马尔',
 '也',
 '以',
 '3',
 '-',
 '1',
 '战胜',
 '了',
 '海伦',
 '芬',
 ',',
 '这样',
 '3',
 '支',
 '球队',
 '同积',
 '72',
 '分',
 ',',
 '阿尔克',
 '马尔',
 '净胜球',
 '最',
 '多排',
 '在',
 '第一',
 ',',
 '阿贾克斯',
 '次之',
 ',',
 '埃因霍温',
 '被',
 '挤',
 '到',
 '了',
 '第三',
 '。',
 ' ',
 '联赛',
 '只',
 '剩下',
 '最后',
 '一轮',
 ',',
 '埃因霍温',
 '就算',
 '在',
 '主场',
 '战胜',
 '维特',
 '斯',
 ',',
 '能否',
 '夺冠',
 ',',
 '也',
 '得',
 '看',
 '别人',
 '脸色',
 ',',
 '只要',
 '另外',
 '两队',
 '有',
 '一支',
 '队伍',
 '获胜',
 ',',
 '联赛',
 '冠军',
 '就',
 '将',
 '拱手相让',
 '。',
 '客观',
 '的',
 '说',
 ',',
 '埃因霍温',
 '夺取',
 '联赛',
 '冠军',
 '的',
 '希望',
 '已经',
 '变得',
 '渺茫',
 '。',
 ' ',
 '中国',
 '球员',
 '孙祥',
 '本场',
 '比赛',
 '没',
 '能',
 '进入',
 '球队',
 '的',
 '18',
 

In [14]:
df_content = pd.DataFrame({'content_S':content_S})
df_content.head(10)

,content_S
0,"[新华网, 武汉, 4, 月, 21, 日电, (, 记者, 郑道, 锦, , 李鹏, 翔..."
1,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日晚, 22, 点, ,, 在..."
2,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 济南, 消息, , 同..."
3,"[搜狐, 体育讯, , 作为, 丁俊晖, (, 丁俊晖, 博客, ,, 丁俊晖, 说, 吧..."
4,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 22, :, 00, ,..."
5,"[搜狐, 体育讯, , 雪藏, 杰拉德, 的, 利物浦, (, 利物浦, 新闻, ,, 利..."
6,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, ,, 2007, 斯诺克..."
7,"[[, 点击, 图片, 进入, 下, 一页, ], , picture, , 北京, 时..."
8,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 22, 日, 凌晨, ,, 2007,..."
9,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 22, :, 00, ,..."


In [8]:
stopwords = pd.read_csv('data/stopwords.txt', index_col=False, sep='\t', quoting=3, names=['stopword'], encoding='utf-8')
stopwords.tail(10)

,stopword
1883,ｎｇ昉
1884,｛
1885,｛－
1886,｜
1887,｝
1888,｝＞
1889,～
1890,～±
1891,～＋
1892,￥


In [9]:
def drop_stopwords(contents, stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        contents_clean.append(line_clean)
    return contents_clean, all_words
    
contents = df_content.content_S.values.tolist()
print(len(contents))
stopwords = stopwords.stopword.values.tolist()
print(len(stopwords))
contents_clean, all_words = drop_stopwords(contents, stopwords)

953
1893


In [13]:
df_content = pd.DataFrame({'contents_clean':contents_clean})
df_content.head(10)

,contents_clean
0,"[新华网, 武汉, 4, 月, 21, 日电, (, 记者, 郑道, 锦, , 李鹏, 翔..."
1,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日晚, 22, 点, ,, 在..."
2,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 济南, 消息, , 同..."
3,"[搜狐, 体育讯, , 作为, 丁俊晖, (, 丁俊晖, 博客, ,, 丁俊晖, 说, 吧..."
4,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 22, :, 00, ,..."
5,"[搜狐, 体育讯, , 雪藏, 杰拉德, 的, 利物浦, (, 利物浦, 新闻, ,, 利..."
6,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, ,, 2007, 斯诺克..."
7,"[[, 点击, 图片, 进入, 下, 一页, ], , picture, , 北京, 时..."
8,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 22, 日, 凌晨, ,, 2007,..."
9,"[搜狐, 体育讯, , 北京, 时间, 4, 月, 21, 日, 22, :, 00, ,..."


In [15]:
df_all_words = pd.DataFrame({'all_words':all_words})
df_all_words.head()

,all_words
0,新华网
1,武汉
2,4
3,月
4,21
